In [105]:
import numpy as np
import pandas as pd
import backtrader as bt
import matplotlib.pyplot as plt
import datetime
from scipy import stats


In [124]:
data_address = "../data/"
companies = ["AAPL", "AMD", "AMZN", "BBBY", "GOOG", "IBM", "INTC", "MSFT", "NFLX", "TGT"]
log_execution = False
log_any_thing = False
log_trade = False
log_next = False
log_notify_order = False

In [3]:
def get_all_data(compaines, data_address):    
    return [pd.read_csv(data_address +  i + ".csv" ) for i in compaines], [data_address +  i + ".csv" for i in compaines]
def print_value(cerebro):
    print("Value Before Running Algorithm ", cerebro.broker.getvalue())
    cerebro.run()
    print("Cache After Running Algorithm ", cerebro.broker.getvalue())

In [4]:
dfs, addreses = get_all_data(companies, data_address)

In [27]:
dfs[0].head(5)

,Date,Open,High,Low,Close,Adj Close,Volume
0,2014-07-14,95.860001,96.889999,95.650002,96.449997,88.398788,42810000
1,2014-07-15,96.800003,96.849998,95.029999,95.320000,87.363113,45477900
2,2014-07-16,96.970001,97.099998,94.739998,94.779999,86.868187,53396300
3,2014-07-17,95.029999,95.279999,92.570000,93.089996,85.319267,57298000
4,2014-07-18,93.620003,94.739998,93.019997,94.430000,86.547401,49988000


In [128]:
class TestStragy(bt.Strategy):
    
    def log(self, txt, dt = None):
        '''
        txt : Message
        dt : Date, if None is given will try and use the time stamp on the 
        '''
        
        global log_any_thing
        
        if not log_any_thing : 
            
            return
        
        dt = dt or self.datas[0].datetime.date(0)
        print("%s, %s" % (dt.isoformat(), txt))
    
    
    def should_trade(self, day):
        
        if day%120 == 0 and day is not 0 :
            
            return True
        
        return False
    
    def should_update(self, day):
        
        if day%120 == 0 :
            
            return True
        
        return False
    
    def should_trade_back(self):
#         trade_back = len(self) - self.last_trading_day == 2
# #         trade_back = trade_back or 
#         if trade_back and self.last_trading_day is not -1 :
#             return True
        if len(self)%120 == 0 and len(self) is not 0 :
            
            return True
        return False
    
    def update(self):
        
        for i, iv in enumerate(self.datas) :
            
            self.returns[i] = np.log2(iv.adjclose[0]/self.last_month[i])
            
            self.last_month[i] = iv.adjclose[0]
            
            self.last_trade = len(self)
            
            
    
    def __init__(self) :
        
        self.last_executed = 0
        self.last_month = np.array([np.nan for i in self.datas])
        self.returns = np.array([np.nan for i in self.datas])
#         0 for not trading, 1 for bought, -1 for sold
        self.positionsM = np.array([0 for i in self.datas])
        self.last_trading_day = -1
        self.k = 5
#         self.traded_prices
    
    def notify_order(self, order):
        '''
        if there is a change in a order, the order is given to this function so that furthur desicions can be made
        '''
        global log_notify_order
        
        if  order.status in [order.Submitted, order.Accepted] : 
            return 
        if order.status in [order.Completed] :
            if order.isbuy():
                if log_notify_order : 
                    self.log('BUY Executed, Price : %2f, Cost : %2f, Comm : %2f' % 
                            (order.executed.price, 
                             order.executed.value, 
                             order.executed.comm))
            elif order.issell():
                if log_notify_order : 
                    self.log('Sell Executed, Price : %2f, Cost : %2f, Comm : %2f' % 
                            (order.executed.price, 
                             order.executed.value, 
                             order.executed.comm))
#             we'll save on which step something was executed
            self.bar_executed = len(self)
        elif order.status in [order.Canceled, order.Margin, order.Rejected] :
            if log_notify_order :     
                self.log('Order Canceled/Margin/Rejected')
        self.order = None
    
    
    def notify_trade(self, trade):
        if not trade.isclosed:
            return
        global trades
        global log_trade
        if log_trade :
            self.log('OPERATION PROFIT, GROSS %.2f, NET %.2f, Value %.2f, Price %.2f' %
                     (trade.pnl, trade.pnlcomm, trade.value, trade.price))
        trades.append(trade)

    def next(self):
        
        global addreses
        global log_next
        
        for i, iv in enumerate(addreses): 
            
            if log_next :
                self.log("Price of "+ iv + " " + str(self.datas[i].adjclose[0]) + " " + str(self.datas[i].close[0]))
            
        
        if self.should_update(len(self)):
            self.update()
        
        if self.should_trade_back():
            if log_next:
                self.log("Ending Trade")
            for i, iv in enumerate(self.positionsM) :
                
                if iv == -1 :
                    if log_next :
                        self.log("Buying Back " + addreses[i]+ " " + str(self.datas[i].adjclose[0])+ " " + str(self.datas[i].close[0]))
                    self.buy(self.datas[i], self.k)
                    
                elif iv == +1 :
                    if log_next : 
                        self.log("Selling Back " + addreses[i]+ " " + str(self.datas[i].adjclose[0])+ " " + str(self.datas[i].close[0]))
                    self.sell(self.datas[i], self.k)
                
                self.positionsM[i] = 0
        
        if self.should_trade(len(self)):
            if log_next :
                self.log("Starting Trade")
            self.last_trading_day = len(self)
            sorted_returns = np.argsort(self.returns)
            n = 2
            
            for i in sorted_returns[:n] :
#                 Ones that have been doing bad : short them
                if log_next :
                    self.log("Selling " + addreses[i]+ " " + str(self.datas[i].adjclose[0])+ " " + str(self.datas[i].close[0]))
                self.sell(self.datas[i], self.k)
                self.positionsM[i] = -1
            
            for i in sorted_returns[-n:] :
#                 Ones that have been doing good : long them
                if log_next :
                    self.log("Buying " + addreses[i]+ " " + str(self.datas[i].adjclose[0])+ " " + str(self.datas[i].close[0]))
                self.buy(self.datas[i], self.k)
                self.positionsM[i] = +1
                
        
                
    

In [179]:
cerebro = bt.Cerebro()
for i in addreses:
    data = bt.feeds.YahooFinanceCSVData(dataname = i, 
                                    # Do not pass values before this date
                                    fromdate=datetime.datetime(2014, 1, 1),
                                    # Do not pass values after this date
                                    todate=datetime.datetime(2019, 1, 1))
    cerebro.adddata(data)
cerebro.addstrategy(TestStragy)
# Setting the commision to 0.1%
cerebro.broker.setcommission(1e-3)


In [180]:
trades = []
print_value(cerebro)

Value Before Running Algorithm  10000.0
Cache After Running Algorithm  12000.647200000007


In [181]:
trades = np.array(trades)

In [182]:
rets = np.array([i.pnl/i.price for i in trades]) 

# Average Raw Return

In [183]:
np.mean(rets)

0.5389433173884088

# The score of a one tailed t-test score

In [184]:
print(stats.ttest_1samp(rets,0)[-1]/2)

0.05375198361193336
